In [5]:
import numpy as np
import matplotlib.pyplot as plt

from lbg_tools import TomographicBin
from u_band_strat import single_col, colors, fig_dir

In [3]:
# Evaluate number densities over a dense grid

m5 = np.linspace(23, 28, 100)
n_u = np.array([TomographicBin("u", m5i).number_density for m5i in m5])
n_g = np.array([TomographicBin("g", m5i).number_density for m5i in m5])
n_r = np.array([TomographicBin("r", m5i).number_density for m5i in m5])
n_i = np.array([TomographicBin("i", m5i).number_density for m5i in m5])
n_z = np.array([TomographicBin("z", m5i).number_density for m5i in m5])

In [ ]:
fig, ax = plt.subplots(figsize=single_col, dpi=150)

ax.plot(m5, n_u, c=colors["u"], label="u")
ax.plot(m5, n_g, c=colors["g"], label="g")
ax.plot(m5, n_r, c=colors["r"], label="r")
ax.plot(m5, n_i, c=colors["i"], label="i")
ax.plot(m5, n_z, c=colors["z"], label="z")

ax.legend(handlelength=1, frameon=False, fontsize=8, loc="lower right")

ax.set(
    yscale="log",
    xlim=(m5.min(), m5.max()),
    ylim=(1e-3, 1e5),
    xlabel="5$\sigma$ depth in detection band",
    ylabel="Number density",
)

ax.scatter(23.48, TomographicBin("u", 23.48).number_density, c="C4", marker="|", s=100)
ax.scatter(24.64, TomographicBin("g", 24.64).number_density, c="C0", marker="|", s=100)
ax.scatter(24.18, TomographicBin("r", 24.18).number_density, c="C2", marker="|", s=100)
ax.scatter(24.18, TomographicBin("i", 24.18).number_density, c="C1", marker="|", s=100)
ax.scatter(23.33, TomographicBin("z", 23.33).number_density, c="C3", marker="|", s=100)

ax.scatter(24.72, TomographicBin("u", 24.72).number_density, c="C4", marker=".", s=50)
ax.scatter(26.00, TomographicBin("g", 26.00).number_density, c="C0", marker=".", s=50)
ax.scatter(25.57, TomographicBin("r", 25.57).number_density, c="C2", marker=".", s=50)
ax.scatter(25.57, TomographicBin("i", 25.57).number_density, c="C1", marker=".", s=50)
ax.scatter(24.63, TomographicBin("z", 24.63).number_density, c="C3", marker=".", s=50)

ax.scatter(26.69, TomographicBin("u", 26.69).number_density, c="C4", marker="*", s=25)
ax.scatter(27.73, TomographicBin("g", 27.73).number_density, c="C0", marker="*", s=25)
ax.scatter(27.69, TomographicBin("r", 27.69).number_density, c="C2", marker="*", s=25)
ax.scatter(27.69, TomographicBin("i", 27.69).number_density, c="C1", marker="*", s=25)
ax.scatter(26.23, TomographicBin("z", 26.23).number_density, c="C3", marker="*", s=25)


fig.savefig(fig_dir / "number_density.pdf", bbox_inches="tight")